In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import multiprocessing as mul
from scipy import stats
import pickle
from scipy.integrate import quad
from corner import corner
import pandas as pd
from scipy.stats import gaussian_kde
import dynesty as dyn

In [2]:
err_grb = []
for i in range(1, 4):
    with open('aefit'+ str(i) +'.out', 'rb') as f:
        temp = f.read().decode('utf-8').split('\n')
        for t in temp:
            if 'Error plotting fits for ' in t:
                err_grb.append(t.split('Error plotting fits for ')[1].split('\n')[0])    

In [3]:
err_grb

['GRB210204A',
 'GRB201216C',
 'GRB200829A',
 'GRB190114C',
 'GRB150314A',
 'GRB140206A',
 'GRB130518A',
 'GRB130427A',
 'GRB090926A',
 'GRB090328',
 'GRB080916C']

In [4]:
grb = err_grb[0]

In [5]:
ncpu = int(mul.cpu_count()*0.75)
grbparam = pd.read_csv(os.getcwd() + '/../data/GRBPARAM.csv', index_col=0)

grbname = grb + '.txt'
grbname_wtht_ext = grbname.replace('.txt','')


arr = np.loadtxt(os.getcwd() + '/asciidataof_fig1/32lag/'+grbname)
data = [arr[:,0], arr[:,1], arr[:,2]]
x = arr[:,0]
y = arr[:,1]
yerr = arr[:,2]

#Properties of GRB
E0 = grbparam[grbname.replace('.txt','')].E0
E0rest = E0
Erest = arr[:,0]    #in keV
z_com = grbparam[grbname.replace('.txt','')].redshift #redshift
H0=67.36 #Hubble constant km -1 Mpc s -1
omega_m = 0.315
omega_l = 1 - omega_m
grb
#MODELS

#NULL model
def nullhp(E, alpha, tau):
    return (1 + z_com)*(tau * ((E ** alpha) - (E0 ** alpha)))
    


def int_z(z_prime, n):
    integ_fn = lambda z: (1+z)**n / np.sqrt(omega_m * (1+z)**3 + omega_l)
    return quad( integ_fn, a=0, b=z_prime)[0]

int_z1 = np.asarray(int_z(z_com, 1))
int_z2 = np.asarray(int_z(z_com, 2))

#LINEAR model
def linearhp(E, logEqg, alpha, tau):
    
    e0qg = (E - E0) / (10 ** logEqg)
    
    return - (e0qg * int_z1)/H0 + nullhp(E, alpha, tau)

#QUADRATIC model
def quadhp(E, logEqg, alpha, tau):
    e0qg = (E**2 - E0 **2) / ((10 ** logEqg)**2)
    
    return -1.5 * (e0qg * int_z2)/H0 + nullhp(E, alpha, tau)
#LOG-LIKELIHOODS
def loglike_null(theta):
    alpha, tau = theta
    
    
    model = nullhp(x,  alpha, tau)
    
    return sum(stats.norm.logpdf(*args) for args in zip(y,model,yerr))
    
    

def loglike_linear(theta):
    logEqg, alpha, tau = theta
    
    model = linearhp(x, logEqg, alpha, tau)
    
    return sum(stats.norm.logpdf(*args) for args in zip(y,model,yerr))

def loglike_quad(theta):
    logEqg, alpha, tau = theta
    
    model = quadhp(x, logEqg, alpha, tau)
    
    return sum(stats.norm.logpdf(*args) for args in zip(y,model,yerr))


alphamin = -1
alphamax = 1
taumin = -10
taumax = 10
logeqmin = 6
logeqmax = 19
#PRIOR DISTRIBUTIONS

def prior_transform_null(theta):
    alpha, tau = theta
    return [(alphamax - alphamin) * alpha + alphamin, (taumax - taumin) * tau + taumin]

def prior_transform_linear(theta):
    logEqg, alpha, tau = theta
    return [(logeqmax - logeqmin) * logEqg + logeqmin, (alphamax - alphamin) * alpha + alphamin, (taumax - taumin) * tau + taumin]

def prior_transform_quadratic(theta):
    logEqg, alpha, tau = theta
    return [(logeqmax - logeqmin) * logEqg + logeqmin, (alphamax - alphamin) * alpha + alphamin, (taumax - taumin) * tau + taumin]

In [17]:
len(err_grb)

11

In [41]:
fig, ax = plt.subplots(8, 4, figsize=(20, 40))
GRBs = ['GRB210619B', 'GRB210610B', 'GRB210204A', 'GRB201216C', 'GRB200829A', 'GRB200613A', 'GRB190114C', 'GRB180720B', 'GRB180703A', 'GRB171010A', 'GRB160625B', 'GRB160509A', 'GRB150821A', 'GRB150514A', 'GRB150403A', 'GRB150314A', 'GRB141028A', 'GRB140508A', 'GRB140206A', 'GRB131231A', 'GRB131108A', 'GRB130925A', 'GRB130518A', 'GRB130427A', 'GRB120119A', 'GRB100728A', 'GRB091003A', 'GRB090926A', 'GRB090618', 'GRB090328', 'GRB081221', 'GRB080916C']
err_grb = GRBs
for grb in err_grb:
    
    
    grbname = grb + '.txt'
    grbname_wtht_ext = grbname.replace('.txt','')


    arr = np.loadtxt(os.getcwd() + '/asciidataof_fig1/32lag/'+grbname)
    data = [arr[:,0], arr[:,1], arr[:,2]]
    x = arr[:,0]
    y = arr[:,1]
    yerr = arr[:,2]

    #Properties of GRB
    E0 = grbparam[grbname.replace('.txt','')].E0
    E0rest = E0
    Erest = arr[:,0]    #in keV
    z_com = grbparam[grbname.replace('.txt','')].redshift #redshift
    threesamplers = ['_quadratic_sampler.pkl', '_linear_sampler.pkl', '_null_sampler.pkl']
    sampler0 = dyn.utils.restore_sampler('./outputs/sampler_saves/' + grb + threesamplers[0])
    sampler1 = dyn.utils.restore_sampler('./outputs/sampler_saves/' + grb + threesamplers[1])
    sampler2 = dyn.utils.restore_sampler('./outputs/sampler_saves/' + grb + threesamplers[2])
    results0 = sampler0.results
    results1 = sampler1.results
    results2 = sampler2.results

    #PLOTTING FITS

    nplot = 1000
    E = np.linspace(min(Erest), max(Erest), nplot)
    samples0 = dyn.utils.resample_equal( results0.samples, np.exp(results0.logwt - results0.logz[-1]))
    # samples0 = np.median(samples0, axis=0)
    samples0 = samples0[np.argmax(results0.logl)]

    samples1 = dyn.utils.resample_equal( results1.samples, np.exp(results1.logwt - results1.logz[-1]))
    # samples1 = np.median(samples1, axis=0)
    samples1 = samples1[np.argmax(results1.logl)]

    samples2 = dyn.utils.resample_equal( results2.samples, np.exp(results2.logwt - results2.logz[-1]))
    # samples2 = np.median(samples2, axis=0)
    samples2 = samples2[np.argmax(results2.logl)]
    null_fit = [nullhp(E[i], samples0[0], samples0[1]) for i in range(nplot)]
    liv_lin_fit = [linearhp(E[i], samples1[0], samples1[1], samples1[2]) for i in range(nplot)]
    liv_quad_fit = [quadhp(E[i], samples1[0], samples1[1], samples1[2]) for i in range(nplot)]

    
    
    ax[err_grb.index(grb)//4, err_grb.index(grb)%4].errorbar(Erest, y, yerr, fmt='o', color='black', label='data')
    ax[err_grb.index(grb)//4, err_grb.index(grb)%4].plot(E, null_fit, label='Null fit')
    ax[err_grb.index(grb)//4, err_grb.index(grb)%4].plot(E, liv_lin_fit,label='Linear fit')
    ax[err_grb.index(grb)//4, err_grb.index(grb)%4].plot(E, liv_quad_fit, label='Quadratic fit')
    ax[err_grb.index(grb)//4, err_grb.index(grb)%4].set_xscale('log')
    ax[err_grb.index(grb)//4, err_grb.index(grb)%4].set_ylim(min(y) - max(abs(yerr)), max(y) + max(abs(yerr)))
    ax[err_grb.index(grb)//4, err_grb.index(grb)%4].legend()
    ax[err_grb.index(grb)//4, err_grb.index(grb)%4].set_xlabel('E (keV)\n')
    ax[err_grb.index(grb)//4, err_grb.index(grb)%4].set_ylabel('lag (s)')
    ax[err_grb.index(grb)//4, err_grb.index(grb)%4].set_title(grb + '\n')
    
    

    

plt.tight_layout()
plt.suptitle('Agarwal et al. 2021 fits\n\n', y=1.0)
plt.savefig('./outputs/fits/ERR_fits.png', facecolor='white', format='png')
plt.show()

In [34]:
len(os.listdir('./outputs/fits/'))

23